In [2]:
from unicodedata import numeric
import pandas
from collections import Counter

numerical_variables = []
categorical_variables = []
with open('/Downloads/employee_attrition.csv','r',encoding='utf-8-sig') as f:
    lines = f.readlines()
    header = lines[0].split(',')
    values = lines[1].split(',')

count = 0
for column in values:
    if(column.strip().isdigit()):
        numerical_variables.append(header[count].strip())
    else:
        categorical_variables.append(header[count].strip())
    count += 1

f.close()



In [3]:
if('Attrition' in numerical_variables):
    numerical_variables.remove('Attrition')
if('Attrition' in categorical_variables):
     categorical_variables.remove('Attrition')

df = pandas.read_csv('/Downloads/employee_attrition.csv')

zero_variance_numerical_variables = []
for column in numerical_variables:
    if(df[column].std() == 0.0):
        zero_variance_numerical_variables.append(column)

zero_variance_categorical_variables = []
high_cardinality_categorical_variables = []
for column in categorical_variables:
    if(len(df[column].unique()) == 1):
         zero_variance_categorical_variables.append(column)

    elif(len(df[column].unique()) >= 200):
        high_cardinality_categorical_variables.append(column)

df.drop(zero_variance_categorical_variables,axis='columns',inplace=True)
df.drop(zero_variance_numerical_variables,axis='columns',inplace=True)
df.drop(high_cardinality_categorical_variables,axis='columns',inplace=True)

for column in zero_variance_numerical_variables:
    numerical_variables.remove(column)

for column in zero_variance_categorical_variables:
     categorical_variables.remove(column)

for column in numerical_variables:
     mean = df[column].mean()
     variance = df[column].std(ddof=0)
     df[column] = (df[column] - mean)/variance

df = pandas.get_dummies(df, prefix_sep='_', drop_first=False,columns=categorical_variables)

cleanup_nums = {"Attrition": {"Yes": 1, "No": 0}}
df = df.replace(cleanup_nums)
df_class_0 = df[df['Attrition'] == 0]
df_class_1 = df[df['Attrition'] == 1]

In [4]:
count_class_0, count_class_1 = df['Attrition'].value_counts()
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pandas.concat([df_class_0, df_class_1_over], axis=0)
X = df_test_over.loc[:, df_test_over.columns != 'Attrition']
y = df_test_over.loc[:, df_test_over.columns == 'Attrition']


from sklearn.model_selection import train_test_split
from collections import Counter
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [12]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
rm=RandomForestClassifier(n_estimators = 100,random_state=0)
rm.fit(X_train,y_train.values.ravel())
y_pred_rm=rm.predict(X_test)
accuracy_rm=accuracy_score(y_test.values.ravel(),y_pred_rm)

In [13]:
accuracy_rm

0.9648648648648649